In [1]:
## DATA MANIPULATION
import numpy as np, pandas as pd
import pickle

## CLASSIFICATION
from sklearn.base import clone
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

### LOAD DATA AND GET LABELS

In [2]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)
labels = train.columns[2:]

### GloVe vectorizer

In [3]:
## BUILD GLOVE EMBEDDINGS DICTIONARY
embeddings_dict = dict()
f = open(r'../data/glove.6B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    embeddings_dict[word] = vec
f.close()
print('Extracted {} word vectors'.format(len(embeddings_dict)))

Extracted 400000 word vectors


In [4]:
## SKLEARN COMPATIBLE GLOVE VECTORIZER TRANSFORMER 
class gloveVectorizer(object):
    def __init__(self, embeddings_dict):
        self.embeddings_dict = embeddings_dict
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(embeddings_dict[next(iter(embeddings_dict))])

    def fit(self, X, y):
        return self
    
    def sentence2vec(self, s):
        '''
        Input:
        Sentence string

        Transformations:
        Get vector for each word -> Average vectors

        Output:
        Vector for sentence
        '''
        words = s.split()
        M = []
        for w in words:
            try:
                M.append(self.embeddings_dict[w])
            except:
                continue
        M = np.array(M)
        v = M.mean(axis=0)
        if type(v) != np.ndarray:
            return np.zeros(self.dim)
        return v # / np.sqrt((v ** 2).sum())

    def transform(self, X):
        X = X.apply(self.sentence2vec)
        return np.stack(X.values, axis=0)

In [5]:
glove = gloveVectorizer(embeddings_dict)
X_train = glove.transform(train['comment_text'])
X_valid = glove.transform(valid['comment_text'])

### Loop through Logistic Regression, SVM, XGBoost

In [9]:
## CREATE RESULTS TABLE
results = pd.DataFrame(columns=['Label','Accuracy', 'Recall', 'Precision', 'F1', 'Vectorizer', 'model'])

## CREATE MODELS
models = {
    'Logistic Regression': LogisticRegression(solver='saga',class_weight='balanced'),
    'SVM': LinearSVC(class_weight='balanced'),
    'XGBoost': ['see below']}
                      
## LOOP THROUGH MODELS
for m_label, model in models.items():
    for label in labels:
        # clone base model (re-initialize weights)
        if m_label == 'XGBoost':
            model = XGBClassifier(n_estimators=100,
                        scale_pos_weight= sum(train[label]==0) / sum(train[label]==1),
                        n_jobs=-1)
        
        m = clone(model) 

        # Fit model
        m.fit(X_train, train[label])
        
        # Get predictions
        preds = m.predict(X_valid)

        # Evaluate predictions
        acc, prec, recall, f1 = (accuracy_score(valid[label], preds), 
                                precision_score(valid[label], preds), 
                                recall_score(valid[label], preds), 
                                f1_score(valid[label], preds))
        
        # Save results to dataframe
        results = results.append({'Label': label,
                                'Accuracy':acc,
                                'Recall':recall,
                                'Precision':prec,
                                'F1':f1,
                                'Vectorizer':'glove',
                                'model': m_label}, 
                                ignore_index = True)
        
        # print results
        print('{0} Results for {1} comments: Accuracy - {2:.2f}; Precision - {3:.2f}; Recall - {4:.2f}; F1 - {5:.2f}'.format(
                                        m_label,
                                        label, 
                                        acc, 
                                        prec, 
                                        recall,
                                        f1))

Logistic Regression Results for toxic comments: Accuracy - 0.90; Precision - 0.48; Recall - 0.86; F1 - 0.62
Logistic Regression Results for severe_toxic comments: Accuracy - 0.80; Precision - 0.05; Recall - 0.99; F1 - 0.09
Logistic Regression Results for obscene comments: Accuracy - 0.93; Precision - 0.43; Recall - 0.87; F1 - 0.58
Logistic Regression Results for threat comments: Accuracy - 0.99; Precision - 0.15; Recall - 0.44; F1 - 0.22
Logistic Regression Results for insult comments: Accuracy - 0.92; Precision - 0.37; Recall - 0.88; F1 - 0.52
Logistic Regression Results for identity_hate comments: Accuracy - 0.82; Precision - 0.05; Recall - 0.96; F1 - 0.09
SVM Results for toxic comments: Accuracy - 0.90; Precision - 0.50; Recall - 0.85; F1 - 0.63
SVM Results for severe_toxic comments: Accuracy - 0.96; Precision - 0.18; Recall - 0.89; F1 - 0.29
SVM Results for obscene comments: Accuracy - 0.94; Precision - 0.46; Recall - 0.86; F1 - 0.60
SVM Results for threat comments: Accuracy - 0.97

In [10]:
## SAVE RESULTS
results.to_csv('../artifacts/glove.csv', index=False)